In [1]:

import pandas as pd
import csv
from collections import Counter
from tqdm import tqdm


In [46]:
#displays given amount of messages from user in given file

def MessageFilter(readPath, minMessages, maxMessages):
    with open(readPath, mode='r', newline='', encoding='utf-8') as readFile:
        reader = csv.DictReader(readFile)
        userNames = []
        dropList = []
        keepList = []
        
        for row in reader: 
            if 'User' in row:
                userNames.append(row['User']) 

    counts = Counter(userNames)
    
    for name, count in counts.items():
        #print(f"{name}: {count}")
        if count >= minMessages and count <= maxMessages:
            keepList.append(name)
        else:
            dropList.append(name)

    data = pd.read_csv(readPath)
    data.drop(data[data['User'].isin(dropList)].index, inplace=True)
    return data, keepList

#stores the user names that we don't want included 
readPath = 'CleanText.csv'
minMessages = 100
maxMessages = 10000
data, keepList = MessageFilter(readPath, minMessages, maxMessages)


In [3]:
#builds training message pairs with true false label that tells if it's from the same user or not
#will have a rewrite or different version done entirely in the future

def TrainingDataBuilder(data, keepList, writePath, messagesPerRow):
    data.sort_values(by=['User'])
    entries = []
    row = 0
    
    for user in tqdm(keepList, desc="Processing users"):

        #from the same user
        messages = []
        messageCount = 0
        
        while messageCount < messagesPerRow:
            if data.iloc[row]['User'] == user:
                messages.append(data.iloc[row]['Message'])
                messageCount += 1
            if row + 1 >= len(data):
                row = 0
            row += 1

        messages1 = messages[int(messagesPerRow / 2):]
        messages2 = messages[:int(messagesPerRow / 2)]
        fromSameUser = True
        
        entries.append([' '.join(messages1), ' '.join(messages2), fromSameUser])

        
        #from different user
        messages = []
        messageCount = 0
        rowB = row
        while messageCount < messagesPerRow:
            if data.iloc[rowB]['User'] != user:
                messages.append(data.iloc[rowB]['Message'])
                messageCount += 1
            if rowB + 1 >= len(data):
                rowB = 0
            rowB += 1

        messages1 = messages[int(messagesPerRow / 2):]
        messages2 = messages[:int(messagesPerRow / 2)]
        fromSameUser = False
        
        entries.append([' '.join(messages1), ' '.join(messages2), fromSameUser])
        
    with open(writePath, 'w', newline='', encoding='utf-8') as writeFile:
        writer = csv.writer(writeFile)
        #Header
        writer.writerow(['Message1', 'Message2', 'FromSameUser'])
        writer.writerows(entries)

#print(len(data))
messagesPerRow = 50
writePath = 'TrainingData.csv'
#TrainingDataBuilder(data, keepList, writePath, messagesPerRow)